<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 521 Files and Databases - Fall 2019</h1>
<h1 style="font-size: 250%;">Problem Set #4</h1>
<h1 style="font-size: 200%;">Version 1.1</h1>
<h1>Issued Tuesday 10/29/2019; Due Thursday 11/07/2019</h1>
</div>

**Homework Information:** The answer cells in this notebook contain metadata that enable it to be automatically parsed. Thus, you must enter your answers *only* into the cells provided. Do not delete any answer cell and/or add a new one to replace it. If you do so by accident, download a new copy of the problem set's notebook and transfer your answers into it. You can introduce new 'coding' cells in an effort to test approaches to problems, however, these introduced cells will not be considered when grading.

Finally, some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be accepted, but your lowest problem-set grade will be dropped. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Do not make any edits in this cell!
class Student:
    first = "Adam"
    last = "Alston"
    onyen = "aalston9"
    pid = "730015776"

By default, SQL only builds ISAM-tree index. In the following questions,
you will need to write Python3 programs to simulate a hash index on the *Game* table
in the <a href="http://csbio.unc.edu/mcmillan/Media/NFL.db" download>NFL.db</a> database.
A function to get the pageid for records in *Game* table is
provided:

<pre>
    def getGamePageId(htid, year):
        htid, year = str(htid), str(year)[3:]
        return int(htid + year)
</pre>

Suppose each *(htid, vtid, pageId)* tuple is 12 bytes and a single page can store at most 340
*(htid, vtid, pageId)* pairs. The hash index is on *(htid, vtid)* and the hash function
is (3*htid+vtid) % 32

---
**Problem 1:** How many buckets will overflow?

In [1]:
# (3*htid+vtid) % 32 == key
# import collections defaultdict type int; any time I create a key it will be initialized to 0

import sqlite3
db = sqlite3.connect("NFL.db")
db.row_factory = sqlite3.Row
cursor = db.cursor()

from collections import defaultdict
pair = defaultdict(int)

cursor.execute("""SELECT htid, vtid FROM Game G""")
for row in cursor.fetchall():
    htid = row['htid']
    vtid = row['vtid']
    
    hashKey = (3*htid+vtid) % 32
    
    if hashKey not in pair:
        # if the line isn't a duplicate, initialize the number of times it has appeared
        pair[hashKey] = 1
    else:
        # the line is a duplicate, increment the amount of times it appears
        pair[hashKey] += 1
        
appearances = list(pair.keys())
appearances.sort()

for line in appearances:
    if pair[line] > 340:
        # print hash followed by the number of times it appears
        print("%d, %d" % (line, pair[line]))

1, 370
5, 368
6, 368
11, 353
16, 386
17, 356
18, 391
20, 355
28, 441
31, 422


In [2]:
import iSQL

query = iSQL.parser("NFL.db")

VkJveChjaGlsZHJlbj0oVGV4dGFyZWEodmFsdWU9dScnLCBkZXNjcmlwdGlvbj11J1NRTDonLCBsYXlvdXQ9TGF5b3V0KGhlaWdodD11JzEwMHB4Jywgd2lkdGg9dSc2MDBweCcpKSwgSEJveCjigKY=


---
**Problem 2:** What is the average number of (htid,vtid, pageId) tuples in buckets?

In [13]:
import sqlite3
db = sqlite3.connect("NFL.db")
db.row_factory = sqlite3.Row
cursor = db.cursor()

from collections import defaultdict
pair = defaultdict(int)

total=0

def getGamePageId(htid, year):
        htid, year = str(htid), str(year)[3:]
        return int(htid + year)

cursor.execute("""SELECT htid, vtid, season FROM Game G""")
for row in cursor.fetchall():
    htid = row['htid']
    vtid = row['vtid']
    year = row['season']
    
    gpid=getGamePageId(htid,year)
    tuple = (htid, vtid)
        
    if hashKey not in pair:
        # if the line isn't a duplicate, initialize the number of times it has appeared
        pair[hashKey] = 1
    else:
        # the line is a duplicate, increment the amount of times it appears
        pair[hashKey] += 1

for row in pair:
    total += pair[row]

print (total/32

10185


---
**Problem 3:** Suppose a page can hold at most 128 records for Game table, on average how many pages you need to fetch to join the game table and team table for a specific pair of teams?

In [68]:
import sqlite3
db = sqlite3.connect("NFL.db")
db.row_factory = sqlite3.Row
cursor = db.cursor()

from collections import defaultdict
pair = defaultdict(int)

pages=0

def getGamePageId(htid, year):
        htid, year = str(htid), str(year)[3:]
        return int(htid + year)

cursor.execute("""SELECT htid, vtid, season FROM Game G""")
for row in cursor.fetchall():
    htid = row['htid']
    vtid = row['vtid']
    year = row['season']
    
    gpid=getGamePageId(htid,year)
    tuple = (htid, vtid)
        
    if tuple not in pair:
        pair[tuple] = set()
        pair[tuple].add(gpid)
    else:
        pair[tuple].add(gpid)
        
print(len(pair))

for (e1,e2) in pair: 
    pages += len(pair[(e1,e2)])

print(pages/float(len(pair)))*2

1051
11.2064700285


In the following problems, you will need to write Python3 programs to
           use extendible hashing for the table from part 1. The hash-index is
           still on *(htid, vtid)* and uses the same hash function in the Game table.

---
**Problem 4:** What is the size of the directory in the extendible hash?

In [161]:
import sqlite3
db = sqlite3.connect("NFL.db")
db.row_factory = sqlite3.Row
cursor = db.cursor()

from collections import defaultdict
pair = {}
tids = {}

cursor.execute("""SELECT htid, vtid FROM Game G""")
for row in cursor.fetchall():
    htid = row['htid']
    vtid = row['vtid']
        
    hashKey = (3*htid+vtid) % 32
    
    if hashKey not in pair:
        pair[hashKey] = 1
        tids[hashKey] = [(htid,vtid)]
    else:
        pair[hashKey] += 1  
        tids[hashKey].append((htid,vtid))        
        
for k,c in pair.iteritems():
    if c > 340:
        lists = list(tids[k])
        tids[k] = []
        for item in lists:
            (htid, vtid) = item
            new = (3*htid+vtid) % 64

            if new not in tids:
                tids[new] = [(htid,vtid)]
            else:
                tids[new].append((htid,vtid))
        
print(len(tids))

# print(pair)
# count=0
# for line in pair:
#     count+= pair[line]

print (max(zip(pair.values(), pair.keys())))
print (min(zip(pair.values(), pair.keys())))

# find which bucket length is equivalent to the key

42


TypeError: 'int' object is not callable

---
**Problem 5:** How many buckets are split?

---
**Problem 6:** How many actual buckets are used in the extendible hash?

---
**Problem 7:** What is the average, maximum and minimum number of (htid,vtid, pageId) tuples in a bucket? Which bucket has the maximum number of keys? Which bucket has the minimum number of keys?

---
**Problem 8:** Are there any empy buckets? If so, how many?

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set.

You can submit it as many times as you would like, but only the last submitted version will be graded. If the last submission was after the due date the final score will be reduced accordingly. 